In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cx_Oracle 
import gzip
import pickle
from datetime import datetime

In [2]:
conn = cx_Oracle.connect('ReportUser/report@10.10.1.7:1521/rproods')
cur = conn.cursor()

In [15]:
query = 'select * from Cent_payment_v where cust_sid=4532193284929818620 '

In [16]:
cur.execute(query)
d = cur.fetchmany()

In [17]:
cols = ['CUST_SID','START_BALANCE','AMOUNT','TXN_TYPE','TXN_REASON','CURRENCY_NAME','CURRENCY_ID','CREATING_INVC_NO','CREATING_INVC_SID','CREATING_CUST_SID','CREATED_DATE','MODIFIED_DATE','DEACTIVATING_INVC_NO','DEACTIVATING_INVC_SID','DEACTIVATING_CUST_SID','DEACTIVATING_DATE','STORE_CODE','STORE_NO','SBS_NO','EMPLOYEE_NAME','CUSTOMER_NAME','RECEIPT_TOTAL','CONTROLLER','CENT_PAYMENT_ID','MISSING_PAYMENT_ID']


In [18]:
data = pd.DataFrame(d, columns=cols)

In [19]:
data.head()

,CUST_SID,START_BALANCE,AMOUNT,TXN_TYPE,TXN_REASON,CURRENCY_NAME,CURRENCY_ID,CREATING_INVC_NO,CREATING_INVC_SID,CREATING_CUST_SID,...,DEACTIVATING_DATE,STORE_CODE,STORE_NO,SBS_NO,EMPLOYEE_NAME,CUSTOMER_NAME,RECEIPT_TOTAL,CONTROLLER,CENT_PAYMENT_ID,MISSING_PAYMENT_ID


In [ ]:
##data.rename(columns={'CREATED_DATE.1':'CUS_CREATED_DATE', 'CREATED_DATE':'INV_CREATED_DATE'}, inplace=True)

In [ ]:
#data.drop(columns='Unnamed: 0', inplace=True)

In [18]:
data.columns

Index(['INVC_SID', 'ITEM_SID', 'QTY', 'PRICE', 'COST', 'INVC_SID', 'SBS_NO',
       'STORE_NO', 'INVC_TYPE', 'CUST_SID', 'CUS_CREATED_DATE', 'FIRST_NAME',
       'LAST_NAME', 'INV_CREATED_DATE', 'LST_SALE_DATE', 'FST_SALE_DATE',
       'LST_SALE_AMT', 'LST_PURCH_DATE'],
      dtype='object')

In [19]:
#update_records(p) Deprecated

last_date = p.sort_values('CREATED_DATE', ascending=True).tail(1)['CREATED_DATE']
#last_date= pd.to_datetime(last_date)
last  = last_date.to_string()
sl = slice(6,16)
ls = last[sl]

ls = datetime.strptime(ls,'%Y-%m-%d')
ls = datetime.strftime(ls,'%d/%m/%Y')
#ls  = '05/May/2019'
#ls = datetime.strptime(ls)
query = "select I.INVC_SID, I.ITEM_SID, I.QTY, I.PRICE, I.COST, v.INVC_SID, v.SBS_NO,  v.STORE_NO, v.INVC_TYPE, v.CUST_SID,v.CREATED_DATE, c.FIRST_NAME, c.LAST_NAME, c.CREATED_DATE, c.LST_SALE_DATE, c.FST_SALE_DATE, c.LST_SALE_AMT, c.LST_PURCH_DATE FROM INVC_ITEM_V I LEFT JOIN INVOICE_V v ON I.INVC_SID=v.INVC_SID LEFT JOIN CUSTOMER_V c ON v.CUST_SID = c.CUST_SID where v.CREATED_DATE > to_date('{}','DD/MM/YYYY')".format(ls)



ls

NameError: name 'p' is not defined

In [20]:
data['EXT_PRICE'] = data.QTY + data.PRICE

In [21]:
data.head()

,INVC_SID,ITEM_SID,QTY,PRICE,COST,INVC_SID,SBS_NO,STORE_NO,INVC_TYPE,CUST_SID,CUS_CREATED_DATE,FIRST_NAME,LAST_NAME,INV_CREATED_DATE,LST_SALE_DATE,FST_SALE_DATE,LST_SALE_AMT,LST_PURCH_DATE,EXT_PRICE
0,-2235808599907283234,-2386876421664658975,1.0,3300.0,2500.00,-2235808599907283234,200,4,0,-2.376408e+18,2010-08-04 13:09:43,AKAWUNMI SEKIRAT,AKANDE,2014-12-01 15:49:53,2010-08-04 13:09:43,2010-08-04 13:09:43,3300.0,2010-08-04 13:09:43,3301.0
1,-2365282266071238215,-2386876430218028334,1.0,4950.0,3403.14,-2365282266071238215,200,9,0,NaN,2014-02-02 15:47:19,None,None,NaT,NaT,NaT,NaN,NaT,4951.0
2,-2365282266071238215,-2386876429231607084,1.0,3500.0,2490.05,-2365282266071238215,200,9,0,NaN,2014-02-02 15:47:19,None,None,NaT,NaT,NaT,NaN,NaT,3501.0
3,-2112739872941416813,-2387016668620424075,2.0,535.0,563.22,-2112739872941416813,200,5,0,NaN,2014-10-10 17:06:40,None,None,NaT,NaT,NaT,NaN,NaT,537.0
4,-2112686077777585479,-2386876398723992670,1.0,4250.0,5555.00,-2112686077777585479,200,5,0,NaN,2014-10-28 17:27:33,None,None,NaT,NaT,NaT,NaN,NaT,4251.0


In [ ]:
data.CUST_SID.dtype

In [22]:
sales_by_cus = data.groupby('CUST_SID').sum().sort_values('EXT_PRICE', ascending=False)

In [23]:
sales_by_cus

,INVC_SID,ITEM_SID,QTY,PRICE,COST,INVC_SID,SBS_NO,STORE_NO,INVC_TYPE,LST_SALE_AMT,EXT_PRICE
CUST_SID,,,,,,,,,,,
-2.312645e+18,-4.435412e+18,-4.773753e+18,2.0,14900.00,9340.11,-4.435412e+18,400.0,12.0,0.0,23100.0,14902.00
-2.376408e+18,-2.109188e+19,-2.144939e+19,9.0,11860.00,5756.48,-2.109188e+19,1800.0,27.0,0.0,25830.0,11869.00
-2.376408e+18,-7.030391e+18,-7.160629e+18,3.0,10720.00,7433.91,-7.030391e+18,600.0,9.0,0.0,16530.0,10723.00
-2.376408e+18,-4.672606e+18,-4.759449e+18,2.0,10000.00,6739.60,-4.672606e+18,400.0,4.0,0.0,18440.0,10002.00
-2.365954e+18,-2.365957e+18,-2.386877e+18,1.0,8450.00,3430.07,-2.365957e+18,200.0,12.0,0.0,120.0,8451.00
-2.376407e+18,-1.171794e+19,-1.193452e+19,6.0,5870.81,3490.87,-1.171794e+19,1000.0,55.0,0.0,15200.0,5876.81
-2.365882e+18,-4.731770e+18,-4.773753e+18,2.0,4030.00,2291.30,-4.731770e+18,400.0,24.0,0.0,8060.0,4032.00
-2.376408e+18,-2.235809e+18,-2.386876e+18,1.0,3300.00,2500.00,-2.235809e+18,200.0,4.0,0.0,3300.0,3301.00
-2.376408e+18,-2.221165e+18,-2.386877e+18,1.0,3150.00,1550.40,-2.221165e+18,200.0,6.0,0.0,4050.0,3151.00


In [ ]:
data.rename(columns={'CREATED_DATE':'INV_CREATED_DATE', 'CREATED_DATE.1':'CUS_CREATED_DATE'}, inplace=True)
data.head(1)

In [ ]:
data.drop(columns=['Unnamed: 0'], inplace = True)

In [ ]:
columns = ["INVC_SID", "ITEM_SID", "QTY", "PRICE", "COST", "INVC_SID", "SBS_NO",  "STORE_NO", "INVC_TYPE", "CUST_SID","CUS_CREATED_DATE", "FIRST_NAME", "LAST_NAME", "INV_CREATED_DATE", "LST_SALE_DATE", "FST_SALE_DATE", "LST_SALE_AMT", "LST_PURCH_DATE"]

In [ ]:
data = pd.DataFrame(s, columns=columns)

In [ ]:
data.rename(columns={9:'CUS_CREATED_DATE'}, inplace=True)

In [ ]:
data.head()


In [ ]:
data.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [ ]:
data.columns

In [ ]:
p  = data.head(100)

In [ ]:
p

In [ ]:
from datetime import datetime


In [ ]:
datetime.today()


In [ ]:
update_records(p)

In [ ]:
first_cust = data.sort_values('FST_SALE_DATE')

In [ ]:
First_50 = first_cust.drop_duplicates('CUST_SID')


In [ ]:
data['EXT_PRICE'] = data['QTY']*data['PRICE']

In [ ]:
data['EXT_COST'] = data['QTY']*data['COST']

In [ ]:
biggest_cust = data.groupby('CUST_SID').EXT_PRICE.sum().sort_values(ascending=False).head(50)

In [ ]:
biggest_cust.to_csv('Top50.csv')

In [ ]:
bg = pd.DataFrame(biggest_cust).reset_index()

In [ ]:
big = bg.join([data['FIRST_NAME'],data['LAST_NAME']], how='left', lsuffix='CUST_SID', rsuffix='CUST_SID')

In [ ]:
cus_no = cur.execute('select * from cust_address_v')

In [ ]:
cust_nos = cus_no.fetchall()

In [ ]:
cust = big_cust.merge(data, how='left', left_on='CUST_SID', right_on='CUST_SID')
